<a href="https://www.kaggle.com/code/rashidrk/lung-cancer-classification-99-on-train-set?scriptVersionId=129448694" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
XData = []
yData = []
XBenign = []


import os
for dirname, _, filenames in os.walk('/kaggle/input/iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset/'):
    for filename in filenames:
        if (filename[-3:] != "txt"):
            category = " ".join(filename.split()[:2])
            img = cv2.imread(os.path.join(dirname, filename))
            img = cv2.resize(img, (512, 512))
            img = img / 255
            
            if(category != "Bengin case"):
                XData.append(img)
                yData.append(category)
                
            else:
                XBenign.append(img)
            
    print(len(filenames))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1
416
120
561


In [2]:
!pip install tqdm #for progress bar

In [3]:
#perform data augmentation
from keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
len(XBenign)

120

In [5]:
np.unique(yData, return_counts= True)

(array(['Malignant case', 'Normal case'], dtype='<U14'), array([561, 416]))

In [6]:
datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.3
)

In [7]:
x = np.array(XBenign)

In [8]:
x.shape

(120, 512, 512, 3)

In [9]:
datagen.fit(x)

In [10]:
len(XBenign)

120

In [11]:
from tqdm import tqdm

In [12]:
for batch, i in zip(datagen.flow(x, batch_size = 1), tqdm(range(510 - 120))):
    if (i >= (510 -120)):
        break
    
    XBenign.append(np.squeeze(batch, axis = 0))

100%|██████████| 390/390 [00:22<00:00, 17.00it/s]


In [13]:
del(x)

In [14]:
len(XBenign)

510

In [15]:
yBenign = ["Bengin case" for i in range(510)]

In [16]:
XData.extend(XBenign)
yData.extend(yBenign)

In [17]:
len(XData), len(yData)

(1487, 1487)

In [18]:
del(yBenign)

In [19]:
XData = np.array(XData)

In [20]:
XData.shape

(1487, 512, 512, 3)

In [21]:
from sklearn.preprocessing import LabelEncoder

In [22]:
encoder = LabelEncoder()
encoder.fit(yData)

LabelEncoder()

In [23]:
yEncoded = encoder.transform(yData)

In [24]:
encoder.inverse_transform([0, 1, 2])

array(['Bengin case', 'Malignant case', 'Normal case'], dtype='<U14')

In [25]:
yEncoded[:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
Xtrain,Xtest,ytrain,ytest = train_test_split(XData, yEncoded, shuffle = True)

In [28]:
Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((1115, 512, 512, 3), (372, 512, 512, 3), (1115,), (372,))

In [29]:
import tensorflow as tf
 
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)) ,
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
 


In [30]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])


In [31]:
callback = tf.keras.callbacks.EarlyStopping(patience = 7)

In [32]:
history = model.fit(Xtrain, ytrain, epochs=15,
                    validation_data=(Xtest, ytest), callbacks = [callback])

Epoch 1/15


/opt/conda/lib/python3.10/site-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


35/35 [==============================] - 392s 11s/step - loss: 1.1494 - accuracy: 0.5525 - val_loss: 0.6440 - val_accuracy: 0.7769
Epoch 2/15
35/35 [==============================] - 383s 11s/step - loss: 0.4665 - accuracy: 0.8350 - val_loss: 0.3257 - val_accuracy: 0.8790
Epoch 3/15
35/35 [==============================] - 382s 11s/step - loss: 0.1758 - accuracy: 0.9453 - val_loss: 0.1344 - val_accuracy: 0.9516
Epoch 4/15
35/35 [==============================] - 383s 11s/step - loss: 0.0373 - accuracy: 0.9883 - val_loss: 0.1081 - val_accuracy: 0.9704
Epoch 5/15
35/35 [==============================] - 383s 11s/step - loss: 0.0317 - accuracy: 0.9910 - val_loss: 0.1675 - val_accuracy: 0.9651
Epoch 6/15
35/35 [==============================] - 383s 11s/step - loss: 0.0124 - accuracy: 0.9955 - val_loss: 0.1415 - val_accuracy: 0.9651
Epoch 7/15
35/35 [==============================] - 385s 11s/step - loss: 0.0162 - accuracy: 0.9973 - val_loss: 0.1267 - val_accuracy: 0.9677
Epoch 8/15
35/35 

In [33]:
model.save("lung cancer.h5")